# Q1. Install the package
To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command mlflow --version and check the output.

What's the version that you have?

In [2]:
!mlflow --version

mlflow, version 2.1.1


# Q2. Download and preprocess the data
We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from here.

Use the script preprocess_data.py located in the folder homework to preprocess the data.

The script will:

* load the data from the folder <TAXI_DATA_FOLDER> (the folder where you have downloaded the data),

* fit a DictVectorizer on the training set (January 2022 data),

* save the preprocessed datasets and the DictVectorizer to disk.

Your task is to download the datasets and then execute this command:

In [3]:
!python3 preprocess_data.py --raw_data_path data/ --dest_path ./output

So what's the size of the saved DictVectorizer file?

* 54 kB

* --> **154 kB** <--

* 54 MB

* 154 MB